In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import layers

from modules.layers import SamplingLayer
from modules.dataset import AntibodyNLFProtDataset
from sklearn.model_selection import train_test_split

from modules.model.VAE import VAE

2024-03-23 23:26:04.482742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 23:26:04.482852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 23:26:04.486381: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 23:26:04.507189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# df_seq = np.load("../../sequence1.npy") # Load anticorps sequences.
x_data, y_data, vector_size, alphabet_size = AntibodyNLFProtDataset.get_data(500)
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

In [4]:
vector_size, alphabet_size

(500, 18)

### Encoder

In [5]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(vector_size, alphabet_size, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = SamplingLayer()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

2024-03-23 23:26:21.792909: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 23:26:21.808356: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 23:26:21.811394: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 500, 18, 1)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 250, 9, 32)           320       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 125, 5, 64)           18496     ['conv2d[0][0]']              
                                                                                                  
 flatten (Flatten)           (None, 40000)                0         ['conv2d_1[0][0]']            
                                                                                            

### Decoder

In [6]:
# Define decoder model. should have output shape (vector_size, alphabet_size, 1)

inputs  = keras.Input(shape=(latent_dim,))
x       = layers.Dense(vector_size // 2 * alphabet_size // 2 * 64, activation="relu")(inputs)
x       = layers.Reshape((vector_size // 2, alphabet_size // 2, 64))(x)
x       = layers.Conv2DTranspose(64, 3, strides=1, padding="same", activation="relu")(x)
x       = layers.Conv2DTranspose(64, 3, strides=2, padding="same", activation="relu")(x)
# x       = layers.Conv2DTranspose(32, 3, strides=2, padding="same", activation="relu")(x)
outputs = layers.Conv2DTranspose(1,  3, padding="same", activation="sigmoid")(x)

decoder = keras.Model(inputs, outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 144000)            432000    
                                                                 
 reshape (Reshape)           (None, 250, 9, 64)        0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 250, 9, 64)        36928     
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 500, 18, 64)       36928     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 500, 18, 1)        577 

## Train the VAE

In [7]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
dataset = np.concatenate([x_train, x_test], axis=0)

In [8]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(dataset, epochs=30, batch_size=128)


Epoch 1/30


2024-03-23 23:26:31.562762: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-03-23 23:26:35.776244: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe2956d2470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-23 23:26:35.776287: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-23 23:26:35.788333: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711236395.935446   67113 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 [==============================] - 13s 290ms/step - loss: 5975.0040 - reconstruction_loss: 5412.2788 - kl_loss: 6.5241
Epoch 2/30
13/13 [==============================] - 1s 93ms/step - loss: 2601.9421 - reconstruction_loss: 2277.3315 - kl_loss: 55.6265
Epoch 3/30
13/13 [==============================] - 1s 92ms/step - loss: 1924.4164 - reconstruction_loss: 1888.0876 - kl_loss: 33.2794
Epoch 4/30
13/13 [==============================] - 1s 93ms/step - loss: 1811.9064 - reconstruction_loss: 1749.8303 - kl_loss: 19.7587
Epoch 5/30
13/13 [==============================] - 1s 93ms/step - loss: 1590.1307 - reconstruction_loss: 1477.6001 - kl_loss: 20.8900
Epoch 6/30
13/13 [==============================] - 1s 93ms/step - loss: 1073.2774 - reconstruction_loss: 635.8199 - kl_loss: 84.5848
Epoch 7/30
13/13 [==============================] - 1s 93ms/step - loss: -8083.9837 - reconstruction_loss: -33000.3906 - kl_loss: 15246.4492
Epoch 8/30
13/13 [==============================] - 1s 91ms/

In [ ]:
vae.save("run/vae-nlf-antibody")

In [14]:
x_data[0].shape

(500, 18)

## References

- <https://keras.io/examples/generative/vae/>